In [1]:
!pip install --upgrade pip
!pip install sentence-transformers langchain chromadb pypdf faiss-cpu  langchain_community scikit-learn matplotlib seaborn numpy mistralai langchain-mistralai langchain_classic

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 30.1 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.6/21.6 MB 48.6 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.7/23.7 MB 119.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 82.8 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 55.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 66.0 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.4/17.4 MB 52.5 MB/s  0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 4.4/4.4 MB 104.5 MB/s  0:00:00
  Created wheel for pypika: filename=pypika-0.48.9-py2.py3-none-any.whl size=53803 sha256=758437f300942263f25e314bb95ec

In [6]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import FAISS,Chroma
from langchain_classic.schema import Document
import os


loader = PyPDFLoader("/content/2022-GROUP-II_20SERVICES-NOTIFN_20FINAL2022122923474120230118102628 (1).pdf")
docs = loader.load()


splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=100)
chunks = splitter.split_documents(docs)
print(f"Loaded {len(docs)} docs, split into {len(chunks)} chunks")
# inspect one
print(chunks[0].page_content[:400])

Loaded 46 docs, split into 300 chunks
TELANGANA STATE PUBLIC SERVICE COMMISSION: HYDERABAD 
NOTIFICATION NO. 28/2022, DATED: 29/12/2022 
GROUP-II SERVICES 
 
(GENERAL RECRUITMENT) 
PARA – I: 
 
1) Applications are invited Online from qualified applicants through the proforma Application to 
be made avail able on Commission’s website (https://www.tspsc.gov.in) to the post s of 
Group-II Services in the State of Telangana. 
 
i)  Submis


In [7]:
print("Creating embeddings... this may take a moment.")
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")

Creating embeddings... this may take a moment.


/tmp/ipython-input-3997694831.py:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.war

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md: 0.00B [00:00, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [31]:
vectorstore = FAISS.from_documents(chunks, embeddings)

In [32]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

In [33]:
from langchain_mistralai import ChatMistralAI

llm = ChatMistralAI(model="mistral-tiny")

In [34]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_classic.chains.combine_documents import create_stuff_documents_chain
from langchain_classic.chains import create_retrieval_chain

prompt = ChatPromptTemplate.from_template("""
Answer the following question based only on the provided context:

<context>
{context}
</context>

Question: {input}
""")

In [35]:
document_chain = create_stuff_documents_chain(llm, prompt)

In [36]:
rag_chain = create_retrieval_chain(retriever, document_chain)

In [38]:
user_query = "What is the qualification of the candidate ?"
response = rag_chain.invoke({"input": user_query})

print("\n--- RESPONSE ---")
print(response["answer"])


--- RESPONSE ---
The candidate, Bhuvan Chandra Mothe, holds a Bachelor of Business Administration degree from Satavahana University with a CGPA of 9.6 and a Master of Computer Applications degree from Loyola Academy Degree and PG College with a CGPA of 9.4.


In [5]:
import os
from getpass import getpass

if not os.getenv("MISTRAL_API_KEY"):
    os.environ["MISTRAL_API_KEY"] = getpass("Enter your Mistral API Key: ")


In [4]:
os.environ["MISTRAL_API_KEY"] = "N59M1vm0sD6xmz8nP5oDyRuCOzioGxHN59M1vm0sD6xmz8nP5oDyRuCOzioGxH4"